In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Other umports
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from scipy.stats import lognorm, loguniform, randint


from sklearn.preprocessing import StandardScaler


# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import RandomizedSearchCV

# metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_curve

# imports

from sklearn.compose import ColumnTransformer, make_column_transformer

from utility import *

## Using this model to make predictions on the kaggle challenge test set

In [29]:
X_set = pd.read_csv('../processed/X_set.csv').drop(['Unnamed: 0'], axis = 1)
X_set.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,age_risk,bp_risk,hdl_risk,ldl_risk,tg_risk,anemic_risk,creatinine_cat,gtp_cat,ast_cat,alt_cat
0,40,165,70,84.0,1.2,1.2,1,1,130,89,...,low risk,high_lvl1,normal,normal,moderate_risk,low risk,normal,normal,normal,low
1,80,160,60,93.0,1.0,1.0,2,2,144,72,...,high risk,high_lvl2,high risk,normal,moderate_risk,low risk,normal,normal,normal,low
2,60,170,70,86.5,0.6,0.7,1,1,117,75,...,high risk,normal,high risk,optimal,moderate_risk,low risk,high,normal,normal,high
3,40,160,50,67.0,0.3,0.4,1,1,116,62,...,low risk,normal,low risk,normal,normal,low risk,low,normal,normal,low
4,40,170,75,89.4,1.0,0.9,1,1,132,94,...,low risk,high_lvl1,high risk,normal,normal,low risk,normal,normal,normal,high


## Load model

In [30]:
import pickle

model = 'lgbm_model.sav'
best_lgbm = pickle.load(open(f'../saved_models/{model}', 'rb'))

In [31]:
best_lgbm

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('drop', 'drop', []),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['systolic', 'Urine protein',
                                                   'age', 'AST', 'LDL', 'Gtp',
                                                   'height(cm)',
                                                   'fasting blood sugar',
                                                   'eyesight(right)',
                                                   'male_probability',
                                                   'eyesight(left)', 'HDL',
                                                   'hemoglobin', 'waist(cm)',
                                                   'relaxation',
                                                   'dental carie...
                                                   'obesity_risk', 'age_risk',
                                                   'bp_risk', 'hdl_risk',
                                                   'ldl_risk', 'tg_risk',
                                                   'anemic_risk',
                                                   'creatinine_cat', 'gtp_cat',
                                                   'ast_cat', 'alt_cat']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['hearing(left)',
                                                   'hearing(right)'])])),
                ('lgbmclassifier',
                 LGBMClassifier(max_depth=8, n_estimators=95, random_state=123,
                                subsample=1))])

In [34]:
pred = best_lgbm.predict(X_set)
pred_proba = best_lgbm.predict_proba(X_set)[:,1]

In [36]:
pred_proba

array([0.60321125, 0.28898522, 0.35749984, ..., 0.43659282, 0.05048339,
       0.02960456])

In [38]:
X_submission = pd.read_csv('../data/test.csv')
X_submission.shape

(106171, 23)

In [42]:
# create folder
path = '../submission/'
if not os.path.exists(path):
    os.makedirs(path)

In [43]:
pd.DataFrame({'id': X_submission['id'], 'smoking': pred_proba}).to_csv('../submission/submission.csv', index=False)

